In [2]:
from sage.graphs.graph_generators import graphs
from itertools import permutations

def hamiltonian_paths(G):
    verts = list(G.vertices())
    n = len(verts)
    paths = []

    for perm in permutations(verts):
        ok = True
        for i in range(n-1):
            if not G.has_edge(perm[i], perm[i+1]):
                ok = False
                break
        if ok:
            # avoid reverse duplicates
            if tuple(reversed(perm)) not in paths:
                paths.append(perm)

    return paths


In [3]:
odd_graphs_7 = []

for G in graphs(7):
    if G.is_connected():
        paths = hamiltonian_paths(G)
        if len(paths) % 2 == 1:
            odd_graphs_7.append((G, paths))

print("Connected 7-vertex graphs with odd number of P7 paths:", len(odd_graphs_7))


Connected 7-vertex graphs with odd number of P7 paths: 122


In [4]:
# ========== p7_odd_2x2_grid PDF generator ==========
# Run in Jupyter with the SageMath kernel

from sage.graphs.graph_generators import graphs
from itertools import permutations
from collections import defaultdict
import tempfile, os
from PIL import Image
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch

# ------------------ helper: count Hamiltonian paths (undirected, unique) ------------------
def count_hamiltonian_paths(G):
    verts = list(G.vertices())
    n = len(verts)
    cnt = 0
    # count permutations that form a Hamiltonian path; each undirected path counted twice
    for perm in permutations(verts):
        ok = True
        for i in range(n-1):
            if not G.has_edge(perm[i], perm[i+1]):
                ok = False
                break
        if ok:
            cnt += 1
    return cnt // 2

# ------------------ collect graphs with odd p7 ------------------
print("Scanning connected 7-vertex graphs (this may take a little bit)...")
odd_graphs = []   # list of tuples: (G, p7_count, num_aut)
for G in graphs(7):
    if G.is_connected():
        p7 = count_hamiltonian_paths(G)
        if p7 % 2 == 1:
            num_aut = G.automorphism_group().cardinality()
            odd_graphs.append((G, p7, num_aut))

print("Found", len(odd_graphs), "connected 7-vertex graphs with odd number of P7 paths.")

# ------------------ sort by automorphism count (increasing) ------------------
odd_graphs.sort(key=lambda t: t[2])  # sort by num_aut

# ------------------ prepare PDF layout: 2x2 grid per page ------------------
pdf_name = "p7_odd_2x2_grid.pdf"
c = canvas.Canvas(pdf_name, pagesize=letter)
page_w, page_h = letter

# margins and grid params
margin = 0.5 * inch
gap = 0.3 * inch

# tile (box) width/height: two columns and two rows
tile_w = (page_w - 2*margin - gap) / 2.0
tile_h = (page_h - 2*margin - gap) / 2.0

# inside each tile, reserve a small header area (for the two numbers)
header_h = 0.35 * inch
image_box_h = tile_h - header_h - 0.15*inch  # slight padding
image_box_w = tile_w - 0.2*inch

# Keep temp files to delete after saving PDF
temp_files = []

# ------------------ iterate and place graphs ------------------
for idx, (G, p7_count, num_aut) in enumerate(odd_graphs):
    pos_in_page = idx % 4  # 0..3
    page_index = idx // 4

    # if first tile on page, and not first page, new page had been shown already at end of previous iteration.
    if pos_in_page == 0 and idx != 0:
        # start a new page (previous page already finished)
        pass

    # compute column (0 left, 1 right) and row (0 top, 1 bottom)
    col = pos_in_page % 2
    row = pos_in_page // 2  # 0 or 1

    # compute tile origin (bottom-left) coordinates in ReportLab coordinate system
    # top-left tile: row=0 -> y tile top at page_h - margin
    tile_x = margin + col * (tile_w + gap)
    # compute tile_y as bottom of tile
    # row 0 (top row) should have tile's top at page_h - margin; tile bottom = top - tile_h
    if row == 0:
        tile_top = page_h - margin
    else:
        tile_top = page_h - margin - (tile_h + gap)
    tile_y = tile_top - tile_h  # bottom y

    # Draw the header text (centered in the tile top area)
    # We'll draw text at (tile_x + 6pt, tile_top - 12pt) (slight padding)
    header_x = tile_x + 6
    header_y = tile_top - 12

    c.setFont("Helvetica-Bold", 9)
    # Left header: "P7: <count>"
    c.drawString(header_x, header_y, f"P7 count: {p7_count}")
    # Right header aligned to right of tile
    c.setFont("Helvetica-Bold", 9)
    right_header_x = tile_x + tile_w - 6
    # draw right-aligned: compute width of string
    right_text = f"Aut: {num_aut}"
    text_w = c.stringWidth(right_text, "Helvetica-Bold", 9)
    c.drawString(right_header_x - text_w, header_y, right_text)

    # ------------------ create image for the graph and fit inside image box ------------------
    # Step 1: save graph to a temporary PNG (use a larger figure for clarity, will scale down)
    tmpfile = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
    tmpname = tmpfile.name
    tmpfile.close()

    # Save using Sage's plotting (use a moderate figure size to get good resolution)
    # Use a square-ish figsize such that images are readable after scaling
    G.plot(figsize=3).save(tmpname)

    # Step 2: open and scale to fit image_box_w x image_box_h while preserving aspect
    img = Image.open(tmpname)
    w, h = img.size
   
    scale_w = (image_box_w) / w
    scale_h = (image_box_h) / h
    scale = min(scale_w, scale_h, 1.0)
    new_w = int(w * scale)
    new_h = int(h * scale)
    if new_w <= 0: new_w = 1
    if new_h <= 0: new_h = 1
    img_resized = img.resize((new_w, new_h), Image.LANCZOS)

    # save resized temporary file
    small_tmp = tmpname.replace(".png", "_small.png")
    img_resized.save(small_tmp)
    temp_files.append(tmpname)
    temp_files.append(small_tmp)

    # compute position to center image inside tile
    img_x = tile_x + (tile_w - (new_w * (1.0))) / 2.0   
    draw_w = new_w * 0.72  
    if (new_w > 0 and new_h > 0):
        # fit exactly into image_box dims (points), preserving aspect
        img_aspect = new_w / new_h
        box_aspect = image_box_w / image_box_h
        if img_aspect > box_aspect:
            draw_w = image_box_w - 6  # small padding
            draw_h = draw_w / img_aspect
        else:
            draw_h = image_box_h - 6
            draw_w = draw_h * img_aspect
    else:
        draw_w = image_box_w - 6
        draw_h = image_box_h - 6

    # compute final draw position (centered)
    draw_x = tile_x + (tile_w - draw_w) / 2.0
    draw_y = tile_y + 0.08*inch  # small bottom padding inside tile

    # draw the image (ReportLab will load file immediately here)
    c.drawImage(small_tmp, draw_x, draw_y, width=draw_w, height=draw_h, preserveAspectRatio=True, mask='auto')

    # if this was the last tile on the page or last graph, finish page
    if pos_in_page == 3:
        c.showPage()
    else:
        # if last graph overall (not filling page), still need to show page at end
        if idx == len(odd_graphs) - 1:
            c.showPage()

# finalize PDF
c.save()
print("PDF created:", pdf_name)

# cleanup all temporary files
for f in temp_files:
    try:
        os.remove(f)
    except:
        pass

print("Temporary images removed.")


Scanning connected 7-vertex graphs (this may take a little bit)...
Found 122 connected 7-vertex graphs with odd number of P7 paths.
PDF created: p7_odd_2x2_grid.pdf
Temporary images removed.


In [ ]:
import csv

M = A

filename = "matrix.csv"

with open(filename, "w", newline="") as f:
    writer = csv.writer(f)
    try:
        writer.writerows(M.rows())   # works if M is a Sage Matrix
    except AttributeError:
        writer.writerows(M)          # works if M is list of lists

print(f"Saved to {filename}")

In [13]:
from sage.graphs.graph_generators import graphs


G7=odd_graphs

print(len(G7))

# Fixed edge order
edge_order = [
(0,1), (0,2), (0,3), (0,4), (0,5), (0,6),
(1,2), (1,3), (1,4), (1,5), (1,6),
(2,3), (2,4), (2,5), (2,6),
(3,4), (3,5), (3,6),
(4,5), (4,6),
(5,6)
]

def edge_bitstring(G):
    return ''.join('1' if G.has_edge(u,v) else '0' for u,v in edge_order)

# Print ONLY matrix entries
for i, (G, p7, num_aut) in enumerate(G7, start=1):
    print(f"G{i} & {edge_bitstring(G)} & 1 &  & - & 1 \\\\")


90
G1 & 000111001010010001010 & 1 &  & - & 1 \\ 1
G2 & 000111001000011001010 & 1 &  & - & 1 \\ 1
G3 & 000110001010010001011 & 1 &  & - & 1 \\ 1
G4 & 000111001010010001011 & 1 &  & - & 1 \\ 1
G5 & 000101001000011010010 & 1 &  & - & 1 \\ 1
G6 & 000111001100011001010 & 1 &  & - & 1 \\ 1
G7 & 000111001100011001011 & 1 &  & - & 1 \\ 1
G8 & 000111001010011010010 & 1 &  & - & 1 \\ 1
G9 & 000111001010011010001 & 1 &  & - & 1 \\ 1
G10 & 000111001010011010011 & 1 &  & - & 1 \\ 1
G11 & 000111001010010001110 & 1 &  & - & 1 \\ 1
G12 & 000111001000011001110 & 1 &  & - & 1 \\ 1
G13 & 000111001010010001111 & 1 &  & - & 1 \\ 1
G14 & 000111001010011010111 & 1 &  & - & 1 \\ 1
G15 & 000111001100101010011 & 1 &  & - & 1 \\ 1
G16 & 001011001010011000000 & 1 &  & - & 1 \\ 1
G17 & 001011001010011000001 & 1 &  & - & 1 \\ 1
G18 & 001011001110011001000 & 1 &  & - & 1 \\ 1
G19 & 001011001110010001001 & 1 &  & - & 1 \\ 1
G20 & 001011001100011001001 & 1 &  & - & 1 \\ 1
G21 & 001011001100010001011 & 1 &  & - & 1 \\ 

In [5]:
# ===============================================================
# HAMILTONIAN CYCLE (C7) CODE
# Excludes graphs appearing in the earlier Hamiltonian-path script
# Includes full automorphism maps (safe sigma(i) version)
# Produces 2×2 PDF exactly like P7 script
# ===============================================================

from sage.graphs.graph_generators import graphs
from itertools import permutations
import tempfile, os
from PIL import Image
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch


# ===============================================================
# 1. Load P7 graphs from your earlier script
# ===============================================================

# odd_graphs_7 is already defined from your earlier code
# Convert to canonical signatures
p7_graphs = set(G.canonical_label().graph6_string() for G, _ in odd_graphs_7)


# ===============================================================
# 2. Hamiltonian cycle counter
# ===============================================================

def count_hamiltonian_cycles(G):
    verts = list(G.vertices())
    n = len(verts)
    cycles = set()

    for perm in permutations(verts):
        ok = True
        for i in range(n):
            if not G.has_edge(perm[i], perm[(i+1) % n]):
                ok = False
                break
        if not ok:
            continue

        # normalize cycle to avoid orientation/rotation duplicates
        p = list(perm)
        rev = list(reversed(p))
        rots = [tuple(p[i:] + p[:i]) for i in range(n)]
        rots_rev = [tuple(rev[i:] + rev[:i]) for i in range(n)]
        canon = min(rots + rots_rev)

        cycles.add(canon)

    return len(cycles)


# ===============================================================
# 3. Collect odd-C7 graphs that were NOT in P7 list
# ===============================================================

odd_c7_graphs = []  # (G, c7_count, num_aut, aut_maps)

print("Scanning connected 7-vertex graphs for odd C7 (excluding P7)...")

for G in graphs(7):
    if not G.is_connected():
        continue

    canon = G.canonical_label().graph6_string()
    if canon in p7_graphs:
        continue

    c7 = count_hamiltonian_cycles(G)
    if c7 % 2 == 1:
        aut_group = G.automorphism_group()
        num_aut = aut_group.cardinality()

        verts = list(G.vertices())
        aut_maps = []

        if num_aut > 1:
            # safest mapping: sigma(i) always returns image of i
            for sigma in aut_group:
                mapping = { verts[i] : verts[sigma(i)] for i in range(len(verts)) }
                aut_maps.append(mapping)

        odd_c7_graphs.append((G, c7, num_aut, aut_maps))

print("Found", len(odd_c7_graphs), "graphs with odd C7 and NOT in P7 list.")


# ===============================================================
# 4. Sort by automorphism count
# ===============================================================

odd_c7_graphs.sort(key=lambda t: t[2])


# ===============================================================
# 5. PDF generator (exact same layout as P7 script)
# ===============================================================

pdf_name = "c7_odd_2x2_grid.pdf"
c = canvas.Canvas(pdf_name, pagesize=letter)
page_w, page_h = letter

margin = 0.5 * inch
gap = 0.3 * inch

tile_w = (page_w - 2*margin - gap) / 2.0
tile_h = (page_h - 2*margin - gap) / 2.0

header_h = 0.35 * inch
image_box_h = tile_h - header_h - 0.15*inch
image_box_w = tile_w - 0.2*inch

temp_files = []


# ===============================================================
# 6. Draw PDF tiles
# ===============================================================

for idx, (G, c7_count, num_aut, aut_maps) in enumerate(odd_c7_graphs):

    pos = idx % 4
    col = pos % 2
    row = pos // 2

    tile_x = margin + col * (tile_w + gap)
    tile_top = page_h - margin - (row * (tile_h + gap))
    tile_y = tile_top - tile_h

    # header
    header_x = tile_x + 6
    header_y = tile_top - 12

    c.setFont("Helvetica-Bold", 9)
    c.drawString(header_x, header_y, f"C7 count: {c7_count}")

    right_text = f"Aut: {num_aut}"
    text_w = c.stringWidth(right_text, "Helvetica-Bold", 9)
    c.drawString(tile_x + tile_w - text_w - 6, header_y, right_text)

    # add a second line if automorphisms exist
    # --- place under header: print automorphism maps (compact, wrapped) ---
    c.setFont("Helvetica", 7)
    maps_x = header_x
    maps_y_start = header_y - 12
    line_height = 8  # small lines for dense maps
    
    if num_aut > 1 and aut_maps:
        # title
        c.setFont("Helvetica-Bold", 7)
        c.drawString(maps_x, maps_y_start, "Automorphism maps:")
        y = maps_y_start - line_height
    
        c.setFont("Helvetica", 7)
        # print up to a reasonable number of maps / lines to avoid overflow
        max_lines = int((image_box_h) / line_height)  # available vertical lines for maps area
        printed = 0
        for amap in aut_maps:
            if printed >= max_lines:
                c.drawString(maps_x, y, "... (more)")
                break
            # convert mapping dict to short string like 0->2,1->1,...
            s = ", ".join(f"{str(k)}->{str(v)}" for k, v in amap.items())
            # truncate if too long
            max_chars = 60
            if len(s) > max_chars:
                s = s[:max_chars-3] + "..."
            c.drawString(maps_x, y, s)
            y -= line_height
            printed += 1
    else:
        # show placeholder if none
        c.setFont("Helvetica-Oblique", 7)
        c.drawString(maps_x, maps_y_start, "No nontrivial automorphisms stored.")
    

    # image
    tmpfile = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
    tmpname = tmpfile.name
    tmpfile.close()

    G.plot(figsize=3).save(tmpname)

    img = Image.open(tmpname)
    w, h = img.size

    scale = min(image_box_w / w, image_box_h / h)
    new_w = int(w * scale)
    new_h = int(h * scale)

    small_tmp = tmpname.replace(".png", "_small.png")
    img.resize((new_w, new_h), Image.LANCZOS).save(small_tmp)

    temp_files.append(tmpname)
    temp_files.append(small_tmp)

    draw_x = tile_x + (tile_w - new_w) / 2
    draw_y = tile_y + 0.08 * inch

    c.drawImage(small_tmp, draw_x, draw_y, width=new_w, height=new_h,
                preserveAspectRatio=True, mask='auto')

    if pos == 3:
        c.showPage()
    elif idx == len(odd_c7_graphs) - 1:
        c.showPage()

c.save()
print("PDF created:", pdf_name)


# ===============================================================
# 7. Cleanup
# ===============================================================

for f in temp_files:
    try:
        os.remove(f)
    except:
        pass

print("Temporary images removed.")


Scanning connected 7-vertex graphs for odd C7 (excluding P7)...
Found 42 graphs with odd C7 and NOT in P7 list.
PDF created: c7_odd_2x2_grid.pdf
Temporary images removed.


In [1]:
from sage.graphs.graph_generators import graphs
G7=odd_c7_graphs
# Fixed edge order
edge_order = [
(0,1), (0,2), (0,3), (0,4), (0,5), (0,6),
(1,2), (1,3), (1,4), (1,5), (1,6),
(2,3), (2,4), (2,5), (2,6),
(3,4), (3,5), (3,6),
(4,5), (4,6),
(5,6)
]

def edge_bitstring(G):
    return ''.join('1' if G.has_edge(u,v) else '0' for u,v in edge_order)

# Print ONLY matrix entries
for i, (G, p7, num_aut, aut) in enumerate(G7, start=1):
    print(f"G{i} & {edge_bitstring(G)} & 1 &  & - & 1 \\\\")
    

NameError: name 'odd_c7_graphs' is not defined